In [1]:
import random
from typing import Optional, Generator

import psycopg2
from faker import Faker

In [2]:
fake = Faker()
conn = psycopg2.connect(
    dbname='baseless_shop',
    user='like_a_boss',
    password='1234',
    host='localhost',
)
conn.set_session(autocommit=True)
cur = conn.cursor()

In [3]:
def fake_users(n: Optional[int] = 5) -> Generator[tuple[str, str, str], None, None]:
    for i in range(n):
        yield fake.user_name(), fake.password(), fake.ascii_email()

In [4]:
def fake_domains(n: Optional[int] = 5) -> Generator[tuple[str, int, str], None, None]:
    for i in range(n):
        yield fake.domain_name(), random.randint(5, 60), fake.text().split('.')[0]

In [5]:
def add_new_users(n: Optional[int] = None) -> None:
    cur.executemany(
        """
        INSERT INTO users (username, password, email)
        VALUES (%s, %s, %s)
        ON CONFLICT (username) DO NOTHING
        """,
        list(fake_users(n) if n is not None else fake_users()),
    )

In [6]:
def add_new_domains(n: Optional[int] = None) -> None:
    cur.executemany(
        """
        INSERT INTO domains (domain, price, description)
        VALUES (%s, %s, %s)
        ON CONFLICT (domain) DO NOTHING
        """,
        list(fake_domains(n) if n is not None else fake_domains()),
    )

In [7]:
add_new_users(2381)

In [8]:
add_new_domains(619)